# FMLIKH Model: Startup Shear

## Protocol-Specific Context (Multi-Mode)

**FMLIKH startup shear** reveals **multi-timescale relaxation** combined with fractional memory:

1. **Multiple overshoot features**: Each mode $i$ contributes overshoot at different times $t_i \sim \tau_i$
2. **Distributed power-law decay**: Each $\lambda_i$ follows Mittag-Leffler decay with rate $\sim t^{-\alpha_i}$
3. **Composite stress response**: $\sigma(t) = \sum_i \sigma_i(t) + \eta_{\infty}\dot{\gamma}$

**Why this matters**: Single-mode FIKH produces one overshoot. FMLIKH with N=3 can show **early, intermediate, and late overshoots** corresponding to fast, medium, and slow structural modes breaking down at different rates. This captures complex experimental startup curves that single-mode models cannot fit.

> **Physical insight**: Fast modes ($\tau_1 \sim 1$ s) break early → sharp initial overshoot. Slow modes ($\tau_3 \sim 100$ s) break late → secondary hump or extended tail. Fractional kinetics broadens each feature.

> **Handbook:** See [FMLIKH Documentation](../../docs/source/models/fikh/fmlikh.rst) for multi-mode transient analysis.

## Learning Objectives

1. Simulate FMLIKH startup with N=3 modes
2. Observe multiple overshoot features from different timescales
3. Analyze mode-by-mode stress contributions
4. Compare FMLIKH vs single-mode FIKH startup
5. Understand how shared vs per-mode $\alpha$ affects overshoot shapes

## Prerequisites

- NB07: FMLIKH flow curve (parameter calibration)
- NB02: FIKH startup (single-mode baseline)

**Estimated Time:** 4-6 minutes (fast), 15-20 minutes (full)

## 1. Setup

In [ ]:
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os
import sys
import time
import warnings

# Robust path resolution for execution from any directory
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    compute_fit_quality,
    get_fmlikh_param_names,
    load_pnas_startup,
    print_convergence_summary,
    save_fikh_results,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

# FAST_MODE: controlled by environment variable
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

jax, jnp = safe_import_jax()
verify_float64()

print(f"FAST_MODE: {FAST_MODE}")

## 2. Load Data

In [ ]:
GAMMA_DOT = 1.0
time_data, stress_data = load_pnas_startup(gamma_dot=GAMMA_DOT)
strain_data = GAMMA_DOT * time_data

print(f"Data points: {len(time_data)}")
print(f"Shear rate: {GAMMA_DOT} 1/s")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, stress_data, "ko", markersize=5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title(f"Startup Shear Data (γ̇ = {GAMMA_DOT} 1/s)")
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 3. NLSQ Fitting

In [ ]:
N_MODES = 3
model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

t0 = time.time()
model.fit(time_data, stress_data, test_mode="startup", strain=strain_data, method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nMode info:")
for m in model.get_mode_info()["modes"]:
    print(f"  Mode {m['mode']}: G={m['G']:.4g}, τ={m['tau']:.4g}")

In [ ]:
metrics = compute_fit_quality(stress_data, model.predict(time_data, test_mode="startup", strain=strain_data))

fig, ax = plot_nlsq_fit(
    time_data, stress_data, model, test_mode="startup",
    param_names=get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True),
    log_scale=False,
    xlabel="Time [s]", ylabel="Stress [Pa]",
    title=f"FMLIKH Startup Fit ({N_MODES} modes, R$^2$ = {metrics['R2']:.5f})",
    x_pred=time_data,
    strain=strain_data,
)
display(fig)
plt.close(fig)

## 4. Bayesian Inference

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model.parameters.get_value(n) for n in param_names if n in model.parameters}

NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 200, 500, 1

t0 = time.time()
result = model.fit_bayesian(
    time_data, stress_data, test_mode="startup", strain=strain_data,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
print(f"Bayesian time: {time.time() - t0:.1f} s")

In [ ]:
key_params = ["G_0", "G_1", "alpha_structure", "sigma_y0"]
print_convergence_summary(result, key_params)

print("\n### Diagnostic Interpretation")
print("| Metric | Target | Meaning |")
print("|--------|--------|---------|")
print("| **R-hat** | < 1.01 | Chain mixing quality |")
print("| **ESS** | > 400 | Independent samples |")
print("| **Divergences** | < 1% | NUTS quality |")

display_arviz_diagnostics(result, key_params, fast_mode=FAST_MODE)

### Convergence Diagnostics

**Diagnostic Targets:**

| Metric | Target | Meaning |
|--------|--------|---------|
| **R-hat** | < 1.01 | Chain mixing quality |
| **ESS** | > 400 | Independent samples |
| **Divergences** | < 1% | NUTS quality |

## 5. Save Results

In [ ]:
save_fikh_results(model, result, "fmlikh", "startup", param_names)

## Key Takeaways

1. **FMLIKH startup shows multiple overshoot features** from different modes
2. **Fast modes → early sharp overshoot; slow modes → late extended tail**
3. **Fractional kinetics broadens each mode's contribution**
4. **Composite stress captures complex experimental startup curves**
5. **Mode decomposition reveals structural level contributions**
6. **Residual analysis** validates multi-mode transient dynamics

---

## Further Reading

- **[FMLIKH Startup Analysis](../../docs/source/models/fikh/fmlikh.rst#distributed-fractional-kinetics)**: Multi-mode transient equations
- **[Mode Decomposition](../../docs/source/models/fikh/fmlikh.rst#hierarchical-microstructure-interpretation)**: Relating modes to physical structure

### Key References

1. Wei, Y., Solomon, M.J., & Larson, R.G. (2018). "A multimode structural kinetics constitutive equation." *J. Rheol.*, 62(1), 321-342.
2. Jaishankar, A. & McKinley, G.H. (2014). "A fractional K-BKZ constitutive formulation." *J. Rheol.*, 58, 1751-1788.

### Next Steps

**Next**: NB09 (Stress Relaxation) — multi-exponential decay with power-law tails